#### traditional way in python, but it took times...

In [2]:
import pandas as pd
import glob

In [3]:
sample_files = glob.glob('C://Users//Administrator//Documents//code//data//audit_data//sample//SAMPLE_*.csv')
population_files = glob.glob('C://Users//Administrator//Documents//code//data//audit_data//population//VEH_*.csv')

#### make a list of file names

In [14]:
file_names = []
for x in population_files :
    file_name = x.split('_')[2].split('.')[0]
    file_names.append(file_name)
    print(file_name)

20180113
20180203
20180323
20180427
20180506
20180622


#### define a function, for applying to each row in the dataframe (this method will take longer time)

In [8]:
append_list = []
def matching_car_id(x, pop_df):
    id_list = []
    
    for idx, row in pop_df.iterrows():
        #print(row.car)
        if x == row.car :
            if row.car_id not in id_list :
                id_list.append(row.car_id)
                return row.car_id
            else :
                new_row = [row.car, row.car_id]
                append_list.append(new_row)
        else :
            #print('not match !')
            continue

#### this will iterrate over the table, which takes longer time

In [ ]:
for idx, file in enumerate(sample_files):
    # load file
    sample_df = pd.read_csv(file, header=None)
    population_df = pd.read_csv(population_files[idx])
    # setting columns in english (it's just my preference)
    population_df.columns = ['situation', 'car_id', 'car']
    sample_df.columns = ["car"]
    append_list = []
    sample_df['car_id'] = sample_df['car'].apply(lambda x: matching_car_id(x, population_df) )
    sample_df = sample_df.append(pd.DataFrame(append_list), ignore_index=True)
    sample_df.to_csv('C://Users//Administrator//Documents//code//data//audit_data//sample2//SAMPLE_{0}'.format(file_names[idx]), encoding='big5', header=None, index=None)
    print(file_names[idx], "done")
    #print(sample_df)
    print('=======================================================')
    print(append_list)

#### using pandas sql method, faster , USE THIS ONE

In [15]:
for idx, file in enumerate(sample_files):
    # load file
    sample_df = pd.read_csv(file, header=None)
    population_df = pd.read_csv(population_files[idx])
    # setting columns in english (it's just my preference)
    population_df.columns = ['situation', 'car_id', 'car']
    sample_df.columns = ["car"]
    
    df = pd.DataFrame(population_df[population_df.car.isin(list(sample_df.car))].groupby(['car', 'car_id']).size()).reset_index().drop(0, axis=1)
    df.to_csv('C://Users//Administrator//Documents//code//data//audit_data//sample2//SAMPLE_{0}.csv'.format(file_names[idx]), encoding='big5', header=None, index=None)